In [1]:
import re
from lxml import etree

In [2]:
inputFile = '../data/source/nb-allRecords.xml'
outputDir = '../data/xml/nb'
limit = 10
offset = 0

In [3]:
root = etree.parse(inputFile)

In [4]:
collection = root.getroot()

In [5]:
records = root.findall("Record")

In [6]:
def getDateForDateElement(date):
    if not date.text:
        return False
        
    patternCeYear = r'\+\d{4}'
    if re.match(patternCeYear, date.text):
        year = date.text[1:].zfill(4)
        if date.tag == 'FromDate':
            return "%s-01-01" % year
        else:
            return "%s-12-31" % year
    return False

In [7]:
dates = root.xpath("//FromDate|//ToDate")
for date in dates:
    fullDate = getDateForDateElement(date)
    if fullDate:
        date.set("fullDate", fullDate)

In [8]:
def convertSwissGridToLatLong(x, y):
    # https://www.swisstopo.admin.ch/en/maps-data-online/calculation-services/navref.html
    # Example: https://geodesy.geo.admin.ch/reframe/navref?format=json&easting=683195&northing=248031&altitude=NaN&input=lv03&output=etrf93-ed
    import requests
    from string import Template
    urlTemplate = Template("https://geodesy.geo.admin.ch/reframe/navref?format=json&easting=$x&northing=$y&altitude=NaN&input=lv03&output=etrf93-ed")
    url = urlTemplate.substitute(x=x, y=y)
    try:
        response = requests.get(url)
        data = response.json()
        return data
    except:
        print("No connection")
    return False

In [9]:
for record in records[:10]:
    xCoord = record.xpath("DetailData/DataElement[@ElementId='10161']/ElementValue/TextValue")
    yCoord = record.xpath("DetailData/DataElement[@ElementId='10162']/ElementValue/TextValue")
    if len(xCoord) and len(yCoord):
        x = xCoord[0].text
        y = yCoord[0].text
        coordinates = convertSwissGridToLatLong(x, y)
        elemCoord = etree.SubElement(record, "Coordinates")
        elemCoord.set("longitude", coordinates['easting'])
        elemCoord.set("latitude", coordinates['northing'])

In [10]:
for record in records[offset:limit]:
    collection.clear()
    recordId = record.get("Id")
        
    collection.append(record)
    outputFile = "%s/nb-record-%s.xml" % (outputDir, recordId)
    with open(outputFile, 'wb') as f:
        f.write(etree.tostring(collection, xml_declaration=True, pretty_print=True, encoding="UTF-8"))